<a href="https://colab.research.google.com/github/min02yam/R-study/blob/main/20220515_%EB%B6%84%EB%A5%98%EB%B6%84%EC%84%9D_weatherAUS_%ED%95%98%EB%8A%94%EC%A4%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

--------------------------------------------------------------------------
### **2. (분류분석) weatherAUS 데이터 셋을 활용하여**
#### 2-1 데이터 요약 값을 보고 NA값이 10,000개 이상인 열을 제외하고 남은 변수중 NA값이 있는 행을 제거하시오
#### 2-2 그리고 AUS 데이터의 Data변수를 Data형으로 변환하고, 전처리가 된 weatherAUS 데이터를 train(70%), test(30%)로 분할하시오 (set.seed(6789))
####2-3 train 데이터로 RainTomorrow를 예측하는 분류모델을 1개 생성하고, test데이터에 대한 예측값을 csv 파일로 제출 하시오
--------------------------------------------------------------------------

---------------------------**목차**---------------------------
1. 데이터 로드
2. 데이터 전처리 (2-1), (2-2)
3. 모델링
SVM OR LOGISTIC 예정


## 2-1.